In [1]:
def getIndexOfLineItemInRow(row):
    for i in range(len(row)):
        if (row[i] != ''):
            return i;

def checkIfAllColumnHeadersAreBlank(row):
    nullcount = 0;
    for i in range(len(row)):
        if (row[i] == ''):
            nullcount = nullcount + 1;

    if (nullcount == len(row)):
        return True
    else:
        return False;

def formatStringForLineItemDesc(string):
    res = ''.join([i for i in string if not i.isdigit()])
    res = res.replace(".", "")
    res = res.replace("(", "")
    res = res.replace(")", "")
    return res

def isHighestLevelLineItem(string):
    splitString = string.split(".")
    if (len(splitString) > 1 and splitString[0].strip().replace(" ", "").isnumeric() and (
            splitString[1].strip().replace(" ", "").isnumeric() == False)):
        return True;

    return False;

def getSectionIdFromRow(row):
    splitRow = row.split(".");
    sectionid = "";

    for i in range(len(splitRow)):
        if (splitRow[i].replace(" ", "").strip().isnumeric()):
            sectionid = sectionid + splitRow[i].replace(" ", "").strip() + ".";
        else:
            break;

    if (sectionid != ""):
        return sectionid[:-1]

    return sectionid;

def getParentSectionIdFromRow(row):
    splitRow = row.split(".");
    sectionid = "";

    for i in range(len(splitRow)):
        if (splitRow[i].replace(" ", "").strip().isnumeric()):
            sectionid = sectionid + splitRow[i].replace(" ", "").strip() + ".";
        else:
            break;

    if (sectionid != ""):
        sectionid = sectionid[:-1]

    splitSectionId = sectionid.split(".")
    sectionid = "";
    for j in range(len(splitSectionId) - 1):
        sectionid = sectionid + splitSectionId[j] + ".";
    if (sectionid != ""):
        return sectionid[:-1]

    return sectionid;

def getSectionFromRow(row):
    splitString=[];
    for i in range(len(row)):
        splitString=row[i].split(" ");
        for j in range (len(splitString)):
            if splitString[j].upper().strip()=="PART" or splitString[j].upper().strip()=="SECTION":
                return row[i].split(":")[1];
    return "";
print("hello3")

hello3


In [2]:


import csv

import pandas as pd

fileName="akhil_testcsv3.csv";
listOfReportSpecificationRows=[]
reportSpecificationRow=[]
print("hello1")

hello1


In [ ]:
with open('D:/Akhil/Work/DOTS/RegAutomation/Akhil/output/lineitemdescription-'+fileName, 'a',newline='') as output:
    newWriter = csv.writer(output)
    with open('D:/Akhil/Work/DOTS/RegAutomation/Akhil/input/tabula-'+fileName, newline='') as File:
        reader = csv.reader(File)
        columnHeaderStringList=[];
        columnHeadersSplitString=[];
        columnHeaderString="";
        noOfColumns=0;
        columnRow=[]
        previousColumnHeaderRow=[]
        serialNumber=0;
        mapOfSectionIdAndLineItemdesc={};
        sectionHeader="";
        #print ("hello, file name is ", fileName)


        for row in reader:
            tempSectionHeader=getSectionFromRow(row);
            #print ("section header - ", tempSectionHeader)
            if(tempSectionHeader!=''):
                sectionHeader=tempSectionHeader;
            # adding all the wrapped column headers together till a line item is found at index 0 and  index 1 isnt a line item
            colIncrement=1;
         
            if(row[0]==''and (len(row)>1 and len(row[1])>1 and row[1][0].isnumeric())==False):
                print(row[0]+'~'+row[1][0]+'~'+row[1])
                noOfColumns=len(row)-1;
                columnHeaderString="";
                while(colIncrement<=noOfColumns):
                    columnHeaderString=columnHeaderString+row[colIncrement]+"~~";
                    colIncrement=colIncrement+1;

                columnHeaderStringList.append(columnHeaderString)
               
            else:
                #assuming max number of columns is 10
                # appending all the tilda separated column headers to make a single column header row
                if(len(columnHeaderStringList)>0):
                    columnRow=[];
                    columnRow.append('')
                    for j in range(10):
                        columnHeaderString="";
                        for i in range(len(columnHeaderStringList)):
                            columnHeadersSplitString=columnHeaderStringList[i].split("~~")
                            if(j<len(columnHeadersSplitString)):
                                columnHeaderString=columnHeaderString+columnHeadersSplitString[j];
                              

                        columnRow.append(columnHeaderString)
                    # if current line item column header is blank we take the previous line item column header
                    if(checkIfAllColumnHeadersAreBlank(columnRow)):
                        columnRow=previousColumnHeaderRow
                    #inserting the column header row
                    newWriter.writerow(columnRow)
                    previousColumnHeaderRow=columnRow


                # assuming columnRow is perfect we take the index of the row which consists of the line item
                # and loop for the column headers and stamp the line item description to the row
                idxLineItemRow=getIndexOfLineItemInRow(row)

                #adding all line items desc sectionwise
                mapOfSectionIdAndLineItemdesc[getSectionIdFromRow(row[idxLineItemRow])]=formatStringForLineItemDesc(row[idxLineItemRow]).strip()

               
                for r in range(len(columnRow)):
                    #adding empty cell to the row to compensate for the column header
                   
                    if(r>=len(row)):
                        row.append('')
                        # if cell is empty and column header is not blank and if its a line item and if its the highest level line item
                    if(row[r]=='' and columnRow[r]!='' and row[idxLineItemRow][0].isnumeric() and isHighestLevelLineItem(row[idxLineItemRow])):

                        row[r]=sectionHeader+" "+formatStringForLineItemDesc(row[idxLineItemRow]).strip()+" "+formatStringForLineItemDesc(columnRow[r]).strip()

                        # report specification rows
                        serialNumber=serialNumber+1;
                        reportSpecificationRow.append(serialNumber)
                        reportSpecificationRow.append(row[idxLineItemRow].split(".")[0].strip().replace(" ",""))
                        reportSpecificationRow.append(row[r])
                        listOfReportSpecificationRows.append(reportSpecificationRow)
                        reportSpecificationRow=[]
                       
                    elif(row[r]=='' and columnRow[r]!='' and row[idxLineItemRow][0].isnumeric()):

                        sectionIdParent=getParentSectionIdFromRow(row[idxLineItemRow]);
                        row[r]=sectionHeader+" "+formatStringForLineItemDesc(mapOfSectionIdAndLineItemdesc[sectionIdParent]).strip()+" "+formatStringForLineItemDesc(columnRow[r]).strip()+" "+formatStringForLineItemDesc(row[idxLineItemRow]).strip()

                        # report specification rows
                        serialNumber=serialNumber+1;
                        reportSpecificationRow.append(serialNumber)
                        reportSpecificationRow.append(row[idxLineItemRow].split(".")[0].strip().replace(" ",""))
                        reportSpecificationRow.append(row[r])
                        listOfReportSpecificationRows.append(reportSpecificationRow)
                        reportSpecificationRow=[]
                       


                        # write rows into csv
                newWriter.writerow(row)
                columnHeaderStringList=[];
                print("hello2")

In [ ]:
df = pd.DataFrame(listOfReportSpecificationRows, columns = ['ID', 'SECTION','LINE ITEM'])

df.to_csv ('D:/Akhil/Work/DOTS/RegAutomation/Akhil/output/reportspecification-'+fileName, index = None, header=True)
print("hello4")